# <center> Лабораторная 1 (часть 2)

In [2]:
import pandas as pd
import numpy as np

Считайте в переменные tr_mcc_codes, tr_types, transactions и customers_gender_train. В transactions опять  только первые 1000000 строк.

In [3]:
tr_mcc_codes = pd.read_csv('files1\\tr_mcc_codes.csv', sep=';')
tr_types = pd.read_csv('files1\\tr_types.csv', sep=';')
transactions = pd.read_csv('files1\\transactions.csv', sep=',', nrows=1000000)
customers_gender_train = pd.read_csv('files1\\gender_train.csv', sep=',')

#### Задание 5
Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train) с помощью merge. Причём с customers_gender_train необходимо использовать соединение с помощью left, а с оставшимися датафреймами - через inner.
В результате соединения датафреймов должно получиться 999584 строки.

In [4]:
merges = transactions.merge(customers_gender_train, how='left').merge(tr_mcc_codes).merge(tr_types)
print(merges)

        customer_id   tr_datetime  mcc_code  tr_type      amount term_id  \
0          39026145    0 10:23:26      4814     1030    -2245.92     NaN   
1          39026145    6 07:08:31      4814     1030    -5614.79     NaN   
2          39026145    8 07:06:10      4814     1030    -1122.96     NaN   
3          39026145   11 08:49:03      4814     1030    -2245.92     NaN   
4          39026145   11 14:12:08      4814     1030    -2245.92     NaN   
...             ...           ...       ...      ...         ...     ...   
999579     36837532   18 23:38:60      3000     6000  1297824.89     NaN   
999580      6808459  206 19:00:10      3000     6000   124693.24     NaN   
999581     83759123   90 22:40:57      3000     6000   218976.79     NaN   
999582     89619970  104 23:39:35      3000     6000   408307.49     NaN   
999583     89619970  105 23:38:49      3000     6000   204153.74     NaN   

        gender                                    mcc_description  \
0          1.0  Зв

#### Задание 6
Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount). (*)

Представьте ответ в виде вещественного числа, округлённого до двух знаков после запятой, отделив дробную часть точкой в формате "123.45"

##### Пояснения:
(\*) Если в результате для мужчин получились значения [-1,-3,-5], а для женщин [-1,-2,-3],  
то модуль разницы между средними арифметическими -3 и -2 будет равен 1.

(\**) Обратите внимание, что для вычисления модуля разности точных знаний о том,  
какой класc относится к мужчинам, а какой - к женщинам, пока не требуется.

(\***) Округление не нужно производить отдельно по средним тратам женщин и мужчин, а только в самом конце, когда получите значение модуля разницы трат.

In [5]:
print(f"{merges[merges['amount'] < 0].groupby('gender')['amount'].mean().diff().abs().values[1]:.2f}")

32718.05


#### Задание 7
1. Создайте новый столбец - mcc_code+tr_type, сконкатенировав значения из соответствующих столбцов. (\*)
2. Оставьте только наблюдения с отрицательным значением amount. Посчитайте дисперсию и количество наблюдений по категориям для получившегося столбца mcc_code+tr_type (\**)
3. Оставить только те строки, в которых количество наблюдений >= 10. Определите отношение максимальной дисперсии к минимальной. Выведите ответ в виде вещественного числа, округлённого до ближайшего целого в формате "123456" без дробной части.

##### Пояснения:
(\*) Для конкатенации значений в столбцах можно использовать метод .astype(str) для серии и складывать соответствующие серии. 

(\**) Для одновременного подсчета количества наблюдений и дисперсии по категориям можно воспользоваться группировкой и функцией .agg()

In [6]:
# 7.1
merges['mcc_code+tr_types'] = (merges['mcc_code'].astype(str) + merges['tr_type'].astype(str)).astype(int)
print('7.1\n', merges['mcc_code+tr_types'], '\n\n')

# 7.2
observations = merges[merges['amount'] < 0].groupby('mcc_code+tr_types')['amount'].agg(['var', 'count'])
print('7.2\n', observations, '\n\n')

# 7.3
vars = observations[observations['count'] >= 10]['var']
print('7.3\nОтношение макс. дисперсии к мин.: ', vars.max() / vars.min())

7.1
 0         48141030
1         48141030
2         48141030
3         48141030
4         48141030
            ...   
999579    30006000
999580    30006000
999581    30006000
999582    30006000
999583    30006000
Name: mcc_code+tr_types, Length: 999584, dtype: int32 


7.2
                             var  count
mcc_code+tr_types                     
7421110            5.963037e+08     24
7421210            1.472955e+08      3
17111110           4.029687e+10     18
17311010           5.335287e+08      3
17311110                    NaN      1
...                         ...    ...
93991110           3.085348e+08      5
93991200           1.843060e+10     18
93991210           4.044796e+09      6
93991510                    NaN      1
94021110           9.900315e+07      3

[667 rows x 2 columns] 


7.3
Отношение макс. дисперсии к мин.:  9170599.00280999


#### Задание 8:
1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту для каждого клиента (из строго положительных сумм по столбцу amount). 
2. Выведите по 5 наименьших транзакций отдельно для мужчин и для женщин (*)
3. Выведите на экран названия транзакций, которые одновременно встречаются у мужчин и женщин из пункта 2.

##### Пояснения:
(\*) Если максимальные суммы приходов по каким-то типам были равны [1,2,3,4,5,6,7,8], то 5 минимальных из них: [1,2,3,4,5]

In [34]:
# 8.1
tr_max = transactions[transactions['amount'] > 0].sort_values(by='amount').drop_duplicates(subset='customer_id', keep='last')
print('8.1\n', tr_max, '\n\n')

# 8.2
tr_max_gender = tr_max.merge(customers_gender_train, how='left', on='customer_id')
fem = tr_max_gender[tr_max_gender['gender'] == 0][:5]
men = tr_max_gender[tr_max_gender['gender'] == 1][:5]
print(f"8.2\nЖенщины:\n{fem}\nМужчины\n{men}\n\n")

# 8.3
print(f"8.3\nОбщие транзакции:\n{pd.merge(fem['tr_type'], men['tr_type']).drop_duplicates().merge(tr_types)}")

8.1
         customer_id   tr_datetime  mcc_code  tr_type        amount term_id
556749     75010111  300 19:26:05      6010     7030  1.123000e+01   80364
110219     68961483  217 10:21:14      6010     7070  2.245900e+02  888991
665116     88260849  297 10:18:10      6010     7070  2.245900e+02  888993
682616      9060208  334 16:43:37      6010     7070  2.245900e+02  888909
271023      1877899  132 08:56:32      6011     7010  1.122960e+03     NaN
...             ...           ...       ...      ...           ...     ...
734222     77695202  132 14:07:18      6010     7070  6.123489e+07     NaN
162550     82581973  192 18:18:17      6010     7020  6.737747e+07     NaN
123977     40917212   43 17:10:40      6010     7070  6.737747e+07     NaN
102522     19950741  143 14:08:24      6010     7070  1.347549e+08     NaN
308597     26782922  151 11:35:44      6010     7070  1.459845e+08     NaN

[1725 rows x 6 columns] 


8.2
Женщины:
    customer_id   tr_datetime  mcc_code  tr_type   amo